In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import time
from pprint import pprint
import requests
from datetime import date, timedelta, datetime
import json
from pprint import pprint
from tqdm import tqdm
from tqdm import tqdm_notebook
# sqlite Dependencies
# ----------------------------------
# Imports the method used for connecting to DBs
from sqlalchemy import create_engine
# Allow us to declare column types
from sqlalchemy import Column, Integer, String, Text, DateTime, Float, Boolean, ForeignKey
from sqlalchemy.orm import relationship
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session

In [2]:
from nba_api.stats.endpoints import playercareerstats, drafthistory, commonplayerinfo, playerawards

In [3]:
# Not used
# cumestatsplayer,draftcombinedrillresults,playerdashboardbyteamperformance, leagueleaders, leaguedashplayerstats, draftcombinenonstationaryshooting, draftcombinestats, commonallplayers,

-------------

# Open Saved JSON

In [4]:
with open('./_players_all_data.json') as json_file:
    data = json.load(json_file)

In [9]:
#  Player positions
player_positions = pd.read_csv('../../datasets/nba_positions.csv') 

In [12]:
five_year_all_players = pd.read_json(data)
twenty_years_all_players = five_year_all_players[five_year_all_players['SEASON_ID'] > '1999-00']
twenty_years_all_players

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,DREB,REB,AST,STL,BLK,TOV,PF,PTS,YIL,PLAYER_NAME
0,1628605,2019-20,0,1610612763,MEM,26,2,0.0,13.0,4,...,1.0,1.0,0,0.0,0.0,0.0,1,12,2,Dusty Hannahs
1,1629658,2019-20,0,1610612757,POR,21,13,0.0,103.0,15,...,18.0,32.0,4,5.0,0.0,4.0,12,38,1,Jaylen Hoard
2,1629682,2019-20,0,1610612738,BOS,22,11,1.0,119.0,14,...,12.0,12.0,16,10.0,2.0,15.0,13,40,1,Tremont Waters
3,1628407,2019-20,0,1610612766,CHA,24,39,11.0,687.0,85,...,85.0,100.0,52,23.0,2.0,37.0,52,224,3,Dwayne Bacon
4,1629644,2019-20,0,1610612748,MIA,21,5,0.0,26.0,3,...,4.0,5.0,1,2.0,1.0,1.0,2,7,1,KZ Okpala
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721,1051,2000-01,0,1610612750,MIN,34,70,21.0,836.0,75,...,152.0,217.0,24,26.0,49.0,25.0,93,177,5,Dean Garrett
5722,1727,2000-01,0,1610612753,ORL,24,76,1.0,1579.0,223,...,159.0,210.0,51,40.0,15.0,68.0,241,628,3,Pat Garrity
5723,1894,2000-01,0,1610612746,LAC,21,69,9.0,1361.0,225,...,203.0,291.0,82,35.0,9.0,106.0,140,690,2,Corey Maggette
5724,2066,2000-01,0,1610612751,NJN,23,34,0.0,223.0,18,...,31.0,53.0,1,2.0,14.0,4.0,23,46,1,Soumaila Samake


#### Noteable Errors
NaN is applied to position if the player position is not found with player salaries. Will fill NaN afterwards.

In [13]:
twenty_years_all_players = twenty_years_all_players.join(player_positions.set_index('PLAYER_NAME'), on='PLAYER_NAME')
twenty_years_all_players.copy()

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,YIL,PLAYER_NAME,POSITION
0,1628605,2019-20,0,1610612763,MEM,26,2,0.0,13.0,4,...,1.0,0,0.0,0.0,0.0,1,12,2,Dusty Hannahs,NaN
1,1629658,2019-20,0,1610612757,POR,21,13,0.0,103.0,15,...,32.0,4,5.0,0.0,4.0,12,38,1,Jaylen Hoard,NaN
2,1629682,2019-20,0,1610612738,BOS,22,11,1.0,119.0,14,...,12.0,16,10.0,2.0,15.0,13,40,1,Tremont Waters,NaN
3,1628407,2019-20,0,1610612766,CHA,24,39,11.0,687.0,85,...,100.0,52,23.0,2.0,37.0,52,224,3,Dwayne Bacon,NaN
4,1629644,2019-20,0,1610612748,MIA,21,5,0.0,26.0,3,...,5.0,1,2.0,1.0,1.0,2,7,1,KZ Okpala,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5721,1051,2000-01,0,1610612750,MIN,34,70,21.0,836.0,75,...,217.0,24,26.0,49.0,25.0,93,177,5,Dean Garrett,NaN
5722,1727,2000-01,0,1610612753,ORL,24,76,1.0,1579.0,223,...,210.0,51,40.0,15.0,68.0,241,628,3,Pat Garrity,NaN
5723,1894,2000-01,0,1610612746,LAC,21,69,9.0,1361.0,225,...,291.0,82,35.0,9.0,106.0,140,690,2,Corey Maggette,NaN
5724,2066,2000-01,0,1610612751,NJN,23,34,0.0,223.0,18,...,53.0,1,2.0,14.0,4.0,23,46,1,Soumaila Samake,NaN


In [ ]:
### DF for each year. decending
first_year = twenty_years_all_players[twenty_years_all_players['YIL'] == 1]
second_year = twenty_years_all_players[twenty_years_all_players['YIL'] == 2]
third_year = twenty_years_all_players[twenty_years_all_players['YIL'] == 3]
fourth_year = twenty_years_all_players[twenty_years_all_players['YIL'] == 4]
fifth_year = twenty_years_all_players[twenty_years_all_players['YIL'] == 5]

## Plot YIL to Total Points

First Year Total Points per Position

In [44]:
import plotly.express as px

fig = px.scatter(first_year, x="PTS", y="POSITION", log_x=True, hover_name="PLAYER_NAME", hover_data=["PTS"], color="PTS",
                 size='PTS', title="First Year Total Points per Position")
fig.show()

First Year Total Points per Position

In [45]:
import plotly.express as px

fig = px.scatter(second_year, x="PTS", y="POSITION", log_x=True, hover_name="PLAYER_NAME", hover_data=["PTS"], color="PTS",
                 size='PTS', title="Second Year Total Points per Position")
fig.show()

In [35]:
## Remove NaN . Will sway values greatly
first_year = first_year.dropna().sort_values(by=['PTS'], ascending=False).drop_duplicates( keep='first')
first_year

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,REB,AST,STL,BLK,TOV,PF,PTS,YIL,PLAYER_NAME,POSITION
2995,201933,2010-11,0,1610612746,LAC,22,82,82.0,3112.0,696,...,989.0,312,63.0,45.0,223.0,252,1845,1,Blake Griffin,PF
4953,2546,2003-04,0,1610612743,DEN,20,82,82.0,2994.0,624,...,498.0,227,97.0,41.0,247.0,225,1725,1,Carmelo Anthony,PF
4863,2544,2003-04,0,1610612739,CLE,19,79,79.0,3120.0,622,...,432.0,465,130.0,58.0,273.0,149,1654,1,LeBron James,SF
3749,201142,2007-08,0,1610612760,SEA,19,80,80.0,2768.0,587,...,348.0,192,78.0,75.0,232.0,122,1624,1,Kevin Durant,SF
2403,203081,2012-13,0,1610612757,POR,22,82,82.0,3167.0,553,...,257.0,531,74.0,19.0,243.0,172,1562,1,Damian Lillard,PG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2102,203459,2013-14,0,1610612757,POR,22,15,0.0,100.0,12,...,9.0,6,2.0,1.0,3.0,10,33,1,Allen Crabbe,G
2034,203318,2013-14,0,1610612764,WAS,23,11,1.0,109.0,11,...,20.0,7,6.0,1.0,9.0,7,32,1,Glen Rice,SF
5568,2072,2000-01,0,1610612749,MIL,21,6,0.0,35.0,5,...,4.0,1,1.0,0.0,1.0,2,13,1,Michael Redd,SG
2902,201607,2010-11,0,1610612740,NOH,27,7,0.0,19.0,0,...,2.0,2,0.0,1.0,1.0,1,3,1,Patrick Ewing,C


In [50]:
test = list(first_year.columns.values)

In [56]:
fig = px.scatter_polar(first_year, r="PTS", theta="POSITION", color="PTS",
                    template="plotly_dark", title="First Year Total Points per Position")
fig.show()

In [27]:
import plotly.express as px
df = px.data.wind()
fig = px.line_polar(first_year, r="PTS", theta="POSITION", color="PTS", line_close=True,
                    color_discrete_sequence=px.colors.sequential.Plasma_r,
                    template="plotly_dark",)
fig.show()

https://plotly.com/python/polar-chart/

In [25]:
# import plotly.express as px
# df = px.data.wind()
# fig = px.line_polar(twenty_years_all_players, r="PTS", theta="POSITION", color="YIL", line_close=True,
#                     color_discrete_sequence=px.colors.sequential.Plasma_r,
#                     template="plotly_dark",)
# fig.show()

In [26]:
# df = px.data.wind()
# df